# Structured Outputs
One of the most powerful features of LLMs is the ability to produce output which conforms to a pre-determined structure.

We define output structures using custom Pydantic types with attribute descriptions and validation logics. See more about using Pydantic types for structured outputs using langchain [here](https://python.langchain.com/docs/concepts/structured_outputs/). This package builds on that functionality by making it easy to request structured output at any point in an exchange.

In [1]:
import pydantic

import sys
sys.path.append('../src/')

import simplechatbot
from simplechatbot.openai_chatbot import OpenAIChatBot

In [2]:

# optional: use this to grab keys from a json file rather than setting system variables
keychain = simplechatbot.APIKeyChain.from_json_file('../keys.json')

chatbot = OpenAIChatBot.new('gpt-4o-mini', api_key=keychain['openai'])
print(chatbot)

OpenAIChatBot(model_type=ChatOpenAI, model_name="gpt-4o-mini", tools=ToolLookup(tools={}))


In [3]:
chatbot.history

[]

In [4]:
chatbot.stream('Hello, how are you? My name is Devin. Don\'t forget it!').print_and_collect()

Hello, Devin! I'm doing well, thank you. How can I assist you today?

ChatResult(content=Hello, Devin! I'm doing well, thank you. How can I assist you today?, tool_calls=[])

In [5]:
print(chatbot.history.get_buffer_string())

Human: Hello, how are you? My name is Devin. Don't forget it!
AI: Hello, Devin! I'm doing well, thank you. How can I assist you today?


In [6]:
class ResponseFormatter(pydantic.BaseModel):
    """Always use this tool to structure your response to the user."""
    answer: str = pydantic.Field(description="The answer to the user's question.")
    followup_question: str = pydantic.Field(description="A follow-up question the user could ask.")

chatbot.chat_structured('what is your favorite cat?', output_structure=ResponseFormatter)

StructuredOutputResult(data=answer="I don't have personal preferences, but many people love the Maine Coon for its friendly personality and size, or the Siamese for its striking appearance and vocal nature. What's your favorite type of cat?" followup_question='Do you have a cat, or are you thinking about getting one?')

In [7]:
print(chatbot.history.get_buffer_string())

Human: Hello, how are you? My name is Devin. Don't forget it!
AI: Hello, Devin! I'm doing well, thank you. How can I assist you today?
Human: what is your favorite cat?
AI: {"answer":"I don't have personal preferences, but many people love the Maine Coon for its friendly personality and size, or the Siamese for its striking appearance and vocal nature. What's your favorite type of cat?","followup_question":"Do you have a cat, or are you thinking about getting one?"}


In [8]:
chatbot.stream('I like that they are so cute!').print_and_collect()

Cats are definitely adorable! Their playful antics and unique personalities make them wonderful companions. Do you have a favorite cat breed, or do you just love all cats?

ChatResult(content=Cats are definitely adorable! Their playful antics and unique personalities make them wonderful companions. Do you have a favorite cat breed, or do you just love all cats?, tool_calls=[])

In [9]:
print(chatbot.history.get_buffer_string())

Human: Hello, how are you? My name is Devin. Don't forget it!
AI: Hello, Devin! I'm doing well, thank you. How can I assist you today?
Human: what is your favorite cat?
AI: {"answer":"I don't have personal preferences, but many people love the Maine Coon for its friendly personality and size, or the Siamese for its striking appearance and vocal nature. What's your favorite type of cat?","followup_question":"Do you have a cat, or are you thinking about getting one?"}
Human: I like that they are so cute!
AI: Cats are definitely adorable! Their playful antics and unique personalities make them wonderful companions. Do you have a favorite cat breed, or do you just love all cats?


In [ ]:
chatbot.chat_structured('what is your favorite cat?', output_structure=ResponseFormatter, add_to_history=False)

StructuredOutputResult(data=answer="I don't have personal favorites, but many people adore the Bengal cat for its unique spotted coat and energetic personality. What's your favorite thing about cats?" followup_question='Are you considering adopting a cat soon?')

In [11]:
print(chatbot.history.get_buffer_string())

Human: Hello, how are you? My name is Devin. Don't forget it!
AI: Hello, Devin! I'm doing well, thank you. How can I assist you today?
Human: what is your favorite cat?
AI: {"answer":"I don't have personal preferences, but many people love the Maine Coon for its friendly personality and size, or the Siamese for its striking appearance and vocal nature. What's your favorite type of cat?","followup_question":"Do you have a cat, or are you thinking about getting one?"}
Human: I like that they are so cute!
AI: Cats are definitely adorable! Their playful antics and unique personalities make them wonderful companions. Do you have a favorite cat breed, or do you just love all cats?
